# 作業目標:
    
    建立一個網路模型

# 作業重點:

請修改 Name 中, 自定義的 Layer 名稱

增加一層全連階層

宣告 MODEL API, 分別採用自行定義的 Input/Output Layer

model.summary 查看 Layers stack

In [1]:
from IPython.display import Image

Image(url="https://keras.io/img/multi-input-multi-output-graph.png")

In [2]:
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model

#主要輸入接收新聞標題本身，即一個整數序列（每個整數編碼一個詞）。
#這些整數在1 到10,000 之間（10,000 個詞的詞彙表），且序列長度為100 個詞
#宣告一個 NAME 去定義Input
main_input = Input(shape=(100,), dtype='int32', name='main_input')


# Embedding 層將輸入序列編碼為一個稠密向量的序列，
# 每個向量維度為 512。
# 從上接收main_input作為輸入
x = Embedding(output_dim=512, input_dim=10000, input_length=100, name='embedding_1')(main_input)

# LSTM 層把向量序列轉換成單個向量，
# 它包含整個序列的上下文信息
# 從上接收 x(embedding_1)作為輸入
lstm_out = LSTM(32, name='lstm_1')(x)

Using TensorFlow backend.


In [3]:
#插入輔助損失，使得即使在模型主損失很高的情況下，LSTM 層和Embedding 層都能被平穩地訓練
#LSTM的第一個輸出 aux_output(Dense)
aux_output = Dense(1, activation='sigmoid', name='aux_output')(lstm_out)

In [4]:
#輔助輸入數據與LSTM 層的輸出連接起來，輸入到模型
import keras
aux_input = Input(shape=(5,), name='aux_input')
#lstm_1 與 aux_input 相互組合merge_1
x = keras.layers.concatenate([lstm_out, aux_input], name='merge_1')


# 堆疊多個全連接網路層
x = Dense(64, activation='relu', name='dense_1')(x)
x = Dense(64, activation='relu', name='dense_2')(x)
#作業解答: 新增兩層
x = Dense(64, activation='relu', name='dense_3')(x)
x = Dense(64, activation='relu', name='dense_4')(x)

# 最後添加主要的邏輯回歸層
main_output = Dense(1, activation='sigmoid', name='main_output')(x)

In [5]:
# 宣告 MODEL API, 分別採用自行定義的 Input/Output Layer
model = Model(inputs=[main_input, aux_input], outputs=[main_output, aux_output])

In [6]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', loss_weights={'main_output':1, 'aux_output':0.2})

In [7]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 512)     5120000     main_input[0][0]                 
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 32)           69760       embedding_1[0][0]                
__________________________________________________________________________________________________
aux_input (InputLayer)          (None, 5)            0                                            
__________________________________________________________________________________________________
merge_1 (C

In [8]:
Image(url='https://miro.medium.com/max/770/1*laH0_xXEkFE0lKJu54gkFQ.png')

#### lstm_1(LSTM) 參數 Param # 計算

我們選擇的LSTM為輸出32維度(32個框框)<br>
所以ht 與 Ct 有著相同的維度32<br><br>
輸入x 為 512維<br>
所以每個框框weight參數是 512+32(新的input:512 + 上一個參數傳過來:32)<br>
LSTM 32個框框 總共參數是(512+32) * 32 + 32(bias) 每個框框搭配一個bias<br><br>
但是LSTM 裡面每個參數的計算是由f(忘記), i(寫入與否), g(寫入多少), o(釋放的外部)四個參數來交互計算決定<br>
所以要再乘以4<br>
((512 + 32) * 32 + 32) * 4 = 69760